In [1]:
testo="""Downregulated genes 1 glgB (all0875) AGTataaaACT 5 94 TAATAA 51 2 talB (all2563) AGTcttaacaaACT 8 32 TAAAAT 19 3 galE(all4713) AGTagttgtcaACT 8 28 TAAATT 39 4 petH (all4121) AGTTagaaggtgAACa 10 198 TTATTAA 230 5 prxA (alr4641) ACAGaTcgtaatcaAACTGT 10 160 TAATCA 152 6 dnaK (alr1742) AGTAttaaTtCT 6 292 TAAATT 278 7 glnA (alr2328) AGgTGtgtcatCAACT 10 94 TATTTC 100 8 cpcB (alr0528) AAGTggcaagaAaTT 7 134 TAAAAT 170 9 ndk (alr3402) AGTtaattgttgACT 9 166 TAAACT 185 10 lexA (alr4908) TAGTactaatgttCTA 8 48 TATTTT 55 11 recA (all3272) AGTAtatctgttcTACT 11 67 TTAATT 108 12 tdk (all4708) AGTgttcacaACT 7 90 TAAAAAT 51 13 ssb1 (alr0088) AGTActtatgTACT 8/6 28 TAATAA 33 14 ssb2 (alr7579) TAAGTcattACTTA 4 236 TATAAT 224 Upregulated genes 1 apcE (alr0020) AGTatgtACT 4 225 GAAAAT 190 2 rbcL (alr1524) AGTaaaagcgttaACT 10 218 ACTTAT 205 3 ahpC (alr4404) TtGTtaaaaaccACTA 8 98 TTAAGA """
#extracting the gene names between the brackets
import re
genes=re.findall(r"\((.*?)\)",testo)
print(genes)

['all0875', 'all2563', 'all4713', 'all4121', 'alr4641', 'alr1742', 'alr2328', 'alr0528', 'alr3402', 'alr4908', 'all3272', 'all4708', 'alr0088', 'alr7579', 'alr0020', 'alr1524', 'alr4404']


In [3]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
tags=[]
seqs=[]
file="/home/davide/Downloads/NostocPCC7120refseq.gb"
file="/home/davide/Downloads/NostocPCC7120refseq.gbk"
for record in SeqIO.parse(file, "genbank"):
    for feature in record.features:
        if feature.type=="CDS":
            if feature.qualifiers.get('old_locus_tag',[''])[0] in genes or feature.qualifiers.get('gene',[''])[0]=='ndk':
                old_locus_tag=feature.qualifiers.get('old_locus_tag',[''])[0]
                gene=feature.qualifiers.get('gene',[''])[0]
                locus_tag=feature.qualifiers["locus_tag"][0]
                product=feature.qualifiers.get('product',[''])[0]
                tags.append(old_locus_tag)
                if gene:
                    description=f"old_locus_tag:{old_locus_tag}, gene:{gene}, product:{product}"
                else:
                    description=f"old_locus_tag:{old_locus_tag}, product:{product}"
                seqs.append(
                    SeqRecord(
                        Seq(str(feature.qualifiers["translation"][0])),
                        id=locus_tag,
                        description=description
                    )
                )

In [4]:
with open("/home/davide/anabenalexa.fasta","a") as f:
    SeqIO.write(seqs,f,"fasta")

In [5]:
#use blastp to find the homologous proteins

In [52]:
import subprocess
import os
database_dir="/home/davide/Desktop/genomiChro/database_RefSeq"
anabena="/home/davide/anabenalexa.fasta"
for file in os.listdir(database_dir):
    if file.endswith(".pdb"):
        file=file[:-4]
        nome=file
        file=os.path.join(database_dir,file)
        output_file=f"/home/davide/Desktop/genomiChro/blastanabena/{nome}tab.txt"
        subprocess.run(["blastp", "-query", anabena, "-db", file, "-out", output_file, "-outfmt", "6 qseqid sseqid pident length evalue"])
        

In [1]:
import subprocess
from pathlib import Path
database="/home/davide/Desktop/genomiChro/databases/PCC6803GT"
proteine_ref=Path("/home/davide/Desktop/genomiChro/proteine_RefSeq")
for file in proteine_ref.iterdir():
    output_file=f"/home/davide/Desktop/genomiChro/blastpvsPCC6803/{file.stem}vsPCC6803.txt"
    subprocess.run(["blastp", "-query", file, "-db", database, "-out", output_file, "-outfmt", "6 qseqid stitle pident length evalue"])

In [6]:
import pandas as pd
file="/home/davide/Downloads/provageni.txt"
#open the file and read the lines
with open(file) as f:
    lines=f.read()
lines=lines.split(" ")
#remove
lines=[x for x in lines if x!='Upregulated' and x!='Downregulated' and x!='genes']
lines=[lines[i:i+8]for i in range(0, len(lines), 8)]
lines
df_anabena=pd.DataFrame(lines,columns=["index","gene_omologo","old_locus_tag","motivo_omologo","spacer","upstream","-10region","-10upstream"])
df_anabena.drop(columns=["index"],inplace=True)
df_anabena.set_index("gene_omologo",inplace=True)
#drop allcolumn not gene or motif
df_anabena.drop(columns=["spacer","upstream","-10region","-10upstream"],inplace=True)
#add column Evidenze with the value: Anabaena PCC 7120: sovraespressione+EMSA
df_anabena["Evidenze"]="Anabaena PCC 7120: sovraespressione+EMSA"
df_anabena["Pubblicazione"]="Kumar et al., 2018. https://doi.org/10.1016/j.bbagrm.2018.07.007"
df_anabena

,old_locus_tag,motivo_omologo,Evidenze,Pubblicazione
gene_omologo,,,,
glgB,(all0875),AGTataaaACT,Anabaena PCC 7120: sovraespressione+EMSA,"Kumar et al., 2018. https://doi.org/10.1016/j...."
talB,(all2563),AGTcttaacaaACT,Anabaena PCC 7120: sovraespressione+EMSA,"Kumar et al., 2018. https://doi.org/10.1016/j...."
galE,(all4713),AGTagttgtcaACT,Anabaena PCC 7120: sovraespressione+EMSA,"Kumar et al., 2018. https://doi.org/10.1016/j...."
petH,(all4121),AGTTagaaggtgAACa,Anabaena PCC 7120: sovraespressione+EMSA,"Kumar et al., 2018. https://doi.org/10.1016/j...."
prxA,(alr4641),ACAGaTcgtaatcaAACTGT,Anabaena PCC 7120: sovraespressione+EMSA,"Kumar et al., 2018. https://doi.org/10.1016/j...."
dnaK,(alr1742),AGTAttaaTtCT,Anabaena PCC 7120: sovraespressione+EMSA,"Kumar et al., 2018. https://doi.org/10.1016/j...."
glnA,(alr2328),AGgTGtgtcatCAACT,Anabaena PCC 7120: sovraespressione+EMSA,"Kumar et al., 2018. https://doi.org/10.1016/j...."
cpcB,(alr0258),AAGTggcaagaAaTT,Anabaena PCC 7120: sovraespressione+EMSA,"Kumar et al., 2018. https://doi.org/10.1016/j...."
ndk,(alr3402),AGTtaattgttgACT,Anabaena PCC 7120: sovraespressione+EMSA,"Kumar et al., 2018. https://doi.org/10.1016/j...."


In [13]:
import pathlib
from pathlib import Path
import pandas as pd
blast=Path("/home/davide/Desktop/genomiChro/blastanabena/")
csv=Path("/home/davide/Desktop/genomiChro/csv/")
#iterate in file that ends with tab.txt
for file in blast.iterdir():
    #print file name witjput the extension

    if file.name.endswith("tab.txt"):
        df=pd.read_csv(file,sep="\t",header=None)
        df.set_index(1,inplace=True)
        df[2]= df[2].apply(lambda x: '{:.2f}'.format(x))
        df[4] = df[4].apply(lambda x: '{:.2e}'.format(x))
        df[4]=df[4].astype(float)
        df['info_blastp']="id: " + df[2].astype(str) + ", len: " + df[3].astype(str) + ", evalue: "  + df[4].astype(str)
        df.rename(columns={0:"gene_omologo"},inplace=True)
        df.index.name="protein_id"
        df = df.sort_values(by=4, ascending=True)
        df = df.loc[~df.index.duplicated(keep='first')]
        df.drop(columns=[2,3,4],inplace=True)
        df=df.join(df_anabena,on="gene_omologo",how="left")
        df["gene_omologo"]=df["gene_omologo"]+df["old_locus_tag"]
        df.drop(columns=["old_locus_tag"],inplace=True)
        #order the columns
        df=df[["motivo_omologo","gene_omologo","info_blastp","Evidenze","Pubblicazione"]]

        for tab in csv.iterdir():
            if Path(tab).stem in file.stem:
                df2=pd.read_excel(tab)
                df2=df2.join(df,on="protein_id",how="left")
                df2.to_excel(file.with_suffix(".xlsx"),index=False)

df


,motivo_omologo,gene_omologo,info_blastp,Evidenze,Pubblicazione
protein_id,,,,,
WP_015156652.1,AGgTGtgtcatCAACT,glnA(alr2328),"id: 79.87, len: 472, evalue: 0.0",Anabaena PCC 7120: sovraespressione+EMSA,"Kumar et al., 2018. https://doi.org/10.1016/j...."
WP_106166489.1,AGTagttgtcaACT,galE(all4713),"id: 80.97, len: 331, evalue: 0.0",Anabaena PCC 7120: sovraespressione+EMSA,"Kumar et al., 2018. https://doi.org/10.1016/j...."
WP_015157218.1,AGTaaaagcgttaACT,rbcL(alr1524),"id: 89.29, len: 476, evalue: 0.0",Anabaena PCC 7120: sovraespressione+EMSA,"Kumar et al., 2018. https://doi.org/10.1016/j...."
WP_106169039.1,AGTatgtACT,apcE(alr0020),"id: 81.11, len: 1064, evalue: 0.0",Anabaena PCC 7120: sovraespressione+EMSA,"Kumar et al., 2018. https://doi.org/10.1016/j...."
WP_015153058.1,AGTTagaaggtgAACa,petH(all4121),"id: 72.33, len: 430, evalue: 0.0",Anabaena PCC 7120: sovraespressione+EMSA,"Kumar et al., 2018. https://doi.org/10.1016/j...."
...,...,...,...,...,...
WP_106168867.1,AGTTagaaggtgAACa,petH(all4121),"id: 38.89, len: 18, evalue: 9.4",Anabaena PCC 7120: sovraespressione+EMSA,"Kumar et al., 2018. https://doi.org/10.1016/j...."
WP_106169531.1,AGTgttcacaACT,tdk(all4708),"id: 30.77, len: 39, evalue: 9.4",Anabaena PCC 7120: sovraespressione+EMSA,"Kumar et al., 2018. https://doi.org/10.1016/j...."
WP_219888929.1,AGTActtatgTACT,ssb1(alr0088),"id: 32.69, len: 52, evalue: 9.6",Anabaena PCC 7120: sovraespressione+EMSA,"Kumar et al., 2018. https://doi.org/10.1016/j...."


In [12]:
df2

,Unnamed: 0,gene,product,motivo,location,pathway,ko_brite,protein_id,GO_component,GO_process,GO_function,motivo_omologo,gene_omologo,info_blastp,Evidenze,Pubblicazione
0,DNA riparo e ricombinazione,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,gyrB,DNA topoisomerase (ATP-hydrolyzing) subunit B,TAGATAAATTGCTCTT,[3673:5611](-),NaN,"['ko03032 - DNA replication proteins', 'ko0340...",WP_015152415.1,GO:0009330 - DNA topoisomerase type II (double...,GO:0006265 - DNA topological change [Evidence ...,GO:0003918 - DNA topoisomerase type II (double...,NaN,NaN,NaN,NaN,NaN
2,NaN,recA,recombinase RecA,TAGTATATCTGCACTA,[751:1828](-),['Homologous recombination'],['ko03400 - DNA repair and recombination prote...,WP_015153795.1,GO:0005737 - cytoplasm [Evidence IEA],GO:0006281 - DNA repair [Evidence IEA],GO:0003677 - DNA binding [Evidence IEA],AGTAtatctgttcTACT,recA(all3272),"id: 87.98, len: 341, evalue: 0.0",Anabaena PCC 7120: sovraespressione+EMSA,"Kumar et al., 2018. https://doi.org/10.1016/j...."
3,NaN,NaN,deoxyribodipyrimidine photo-lyase,TAGTACAAGTATACTG,[<0:259](-),NaN,['ko03400 - DNA repair and recombination prote...,WP_146138875.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,DNA polymerase III subunit gamma/tau,TAGTGCATTAGTACTA,[1325:3389](+),"['DNA replication', 'Mismatch repair', 'Homolo...","['ko03032 - DNA replication proteins', 'ko0340...",WP_106169649.1,NaN,NaN,NaN,AGTgttcacaACT,tdk(all4708),"id: 24.81, len: 129, evalue: 8.3",Anabaena PCC 7120: sovraespressione+EMSA,"Kumar et al., 2018. https://doi.org/10.1016/j...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,NaN,NaN,site-specific integrase,TAGTACAAAAGTTCTA,[12622:13816](-),NaN,[],WP_106166425.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
222,NaN,NaN,PAS domain S-box protein,TATATCAAGTTTACTA,[9247:11044](-),NaN,[],WP_106167739.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223,NaN,NaN,IS630 family transposase,AAGAACAGCTTCTCTT,[1994:3054](-),NaN,[],NaN,NaN,NaN,GO:0004803 - transposase activity [Evidence IEA],NaN,NaN,NaN,NaN,NaN
224,NaN,NaN,hypothetical protein,TAGTACATTCAAACTG,[2674:2851](+),NaN,[],WP_158631965.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
file="/home/davide/Downloads/CCMEE_29.csv"
df2=pd.read_csv(file)
df2=df2.join(df,on="protein_id",how="left")
df2

,Unnamed: 0,gene,product,motivo,location,pathway,ko_brite,protein_id,GO_component,GO_process,GO_function,motivo_omologo,gene_omologo,info_blastp,Evidenze,Pubblicazione
0,DNA riparo e ricombinazione,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,lexA,transcriptional repressor LexA,TAGTACGAATGTTCTA,[2936606:2937215](+),NaN,"['ko01002 - Peptidases and inhibitors', 'ko034...",WP_250121220.1,NaN,GO:0006282 - regulation of DNA repair [Evidenc...,GO:0004252 - serine-type endopeptidase activit...,NaN,NaN,NaN,NaN,NaN
2,NaN,recA,recombinase RecA,TAGTATATCTGCACTA,[3253689:3254766](+),['Homologous recombination'],['ko03400 - DNA repair and recombination prote...,WP_250121493.1,GO:0005737 - cytoplasm [Evidence IEA],GO:0006281 - DNA repair [Evidence IEA],GO:0003677 - DNA binding [Evidence IEA],NaN,NaN,NaN,NaN,NaN
3,NaN,radA,DNA repair protein RadA,TATTTCAGCCTTTCTT,[895445:896960](+),NaN,['ko03400 - DNA repair and recombination prote...,WP_250124411.1,NaN,GO:0006281 - DNA repair [Evidence IEA],GO:0005524 - ATP binding [Evidence IEA],NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,ATP-dependent RecD-like DNA helicase,TAGTAAATTTGTACTA,[3439950:3442179](-),['Homologous recombination'],['ko03400 - DNA repair and recombination prote...,WP_250121656.1,NaN,NaN,GO:0004386 - helicase activity [Evidence IEA],NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,NaN,NaN,IS630 family transposase,CAGTGCAGCCATTCTA,[230448:231480](+),NaN,[],WP_250121189.1,NaN,NaN,GO:0004803 - transposase activity [Evidence IEA],NaN,NaN,NaN,NaN,NaN
182,NaN,NaN,metallothionein,AAGAGTAGTTGCTCTA,[362085:362250](+),NaN,[],WP_250123942.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,NaN,NaN,ssl1498 family light-harvesting-like protein,TATTGTGATTTTTCTA,[147466:147634](-),NaN,[],WP_250126641.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184,NaN,NaN,VOC family protein,TAGAACAACTTCAATG,[1199937:1200360](-),NaN,[],WP_250124683.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
folder_id="1YDMwApF3vLdwxUzBXBZZJWsEqzM0GOoU"
import gspread
gc = gspread.oauth()
worksheet_dict = gc.list_spreadsheet_files(folder_id=folder_id)
from pprint import pprint

#from dict to list with only key name
worksheet_list = [el['name'] for el in worksheet_dict]

from pathlib import Path
import pandas as pd
blast=Path("/home/davide/Desktop/genomiChro/blastanabena/")
csv=Path("/home/davide/Desktop/genomiChro/csv/")
#iterate in file that ends with tab.txt
for file in blast.iterdir():
    #print file name witjput the extension

    if file.name.endswith("tab.txt"):
        df=pd.read_csv(file,sep="\t",header=None)
        df.set_index(1,inplace=True)
        df[2]= df[2].apply(lambda x: '{:.2f}'.format(x))
        df[4] = df[4].apply(lambda x: '{:.2e}'.format(x))
        df[4]=df[4].astype(float)
        df['info_blastp']="id: " + df[2].astype(str) + ", len: " + df[3].astype(str) + ", evalue: "  + df[4].astype(str)
        df.rename(columns={0:"gene_omologo"},inplace=True)
        df.index.name="protein_id"
        df = df.sort_values(by=4, ascending=True)
        df = df.loc[~df.index.duplicated(keep='first')]
        df.drop(columns=[2,3,4],inplace=True)
        df=df.join(df_anabena,on="gene_omologo",how="left")
        df["gene_omologo"]=df["gene_omologo"]+df["old_locus_tag"]
        df.drop(columns=["old_locus_tag"],inplace=True)
        for x in worksheet_list:
            print(x)
            if x in file.stem:
                sh = gc.open(x)
                #insert in A1 Nan
                worksheet = sh.get_worksheet(1)
                worksheet.update_cell(1,1, 'Nan')
                worksheet.delete_columns(12,14)
                df2=pd.DataFrame(worksheet.get_all_records())
                df2=df2.join(df,on="protein_id",how="left")
                df2=df2.fillna(value="NaN")

                if(len(sh.worksheets())==1):
                    new_wh=sh.add_worksheet(title="con_evidenze_Anabaena",rows=1000,cols=100)
                worksheet = sh.get_worksheet(1)
                new=[df2.columns[-5:].tolist()]+ df2.iloc[:,-5:].values.tolist()
                new=list(map(list, zip(*new)))
                
                worksheet.insert_cols(new,col=5)
                #worksheet.update([df2.columns.values.tolist()] + df2.values.tolist())


    

SAG_2025


GSpreadException: the header row in the worksheet is not unique, try passing 'expected_headers' to get_all_records

In [4]:
file="/home/davide/cdanabena.txt"
with open(file) as f:
    lines=f.read()
lines.split(" ")

['dnaK2@',
 'alr1742',
 'AGTAttaaTtCT',
 '6',
 '292',
 'TAAATT',
 '278',
 '-',
 'YES',
 'Kumar',
 'et',
 'al.,',
 '2018',
 'tufA@',
 'all4337',
 'No',
 'AnLexA',
 'box',
 '-',
 '-',
 'AGATATCAT',
 '25',
 'fus-tufA',
 'NO',
 'Srivastava',
 'et',
 'al.,',
 '2022b',
 'PREPRINT',
 'atpA@',
 'all0005',
 'No',
 'AnLexA',
 'box',
 '-',
 '-',
 'AATTAAAAT',
 '109',
 'all0005all0004',
 'NO',
 'Srivastava',
 'et',
 'al.,',
 '2022a',
 'alr0880',
 'alr0880',
 'TTAGTcctataACTAA',
 '6',
 '124',
 'TAGTAGAAT',
 '34',
 '-',
 'YES',
 'This',
 'study',
 'alr0803@',
 'alr0803',
 'AGTAcataccTAaT',
 '8',
 '390',
 'AATAATTA',
 '259',
 '-',
 'NO',
 'Srivastava',
 'et',
 'al.,',
 '2022b',
 'PREPRINT',
 'alr1827',
 'alr1827',
 'AGTAgttTACT',
 '5',
 '239',
 'TCCTATACC',
 '191',
 '-',
 'YES',
 'This',
 'study',
 'alr1890',
 'alr1890',
 'AGTctaaACT',
 '4',
 '45',
 'GTCTAAACT',
 '45',
 '-',
 'YES',
 'This',
 'study',
 'ccmM',
 'all0865',
 'TCAGTggcaaaacaACTGA',
 '9',
 '64',
 'CGACAAAAT',
 '22',
 '-',
 'YES',
 'This'

In [24]:
sh = gc.open("SAG_2025")
wh = sh.get_worksheet(1)
#insert new cols at pos 4
#pick last 4 columns of df2 and transform to list of lsits
new=[df2.columns[-3:].tolist()]+ df2.iloc[:,-3:].values.tolist()
new=list(map(list, zip(*new)))
wh.insert_cols(new,col=5)

{'spreadsheetId': '1Etovzk6PI8TdT_i15_8Q7m5qZ9TY7-nN2uj6USgCLnU',
 'updates': {'spreadsheetId': '1Etovzk6PI8TdT_i15_8Q7m5qZ9TY7-nN2uj6USgCLnU',
  'updatedRange': 'con_evidenze_Anabaena!D1:F227',
  'updatedRows': 227,
  'updatedColumns': 3,
  'updatedCells': 681}}